In [1]:
import os
import re
import json
import pandas as pd

In [2]:
LIST_FILE = []

for root, dirs, files in os.walk("outputs/tunning/"):
    if files:
        for file in files:
            if file.endswith(".json"):
                LIST_FILE.append(os.path.join(root, file))

In [3]:
LIST_FILE

['outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_043\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_043\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_045\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_045\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_047\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_047\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_050\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_050\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_051\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_BIRD_051\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_DRONE_099\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_DRONE_099\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_DRONE_101\\multiscale_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_DRONE_101\\yolo_det.json',
 'outputs/tunning/iou_0.1_conf_0.1\\V_DRONE_104\\multiscal

In [23]:
MULTISCALE_DATA = []
YOLO_DATA = []

for file in LIST_FILE:
    filename = os.path.basename(file)
    if "multiscale" in filename:
        with open(file, 'r') as f:
            data = json.load(f)
            MULTISCALE_DATA.append(data)
    elif "yolo" in filename:
        with open(file, 'r') as f:
            data = json.load(f)
            YOLO_DATA.append(data)

In [24]:
data = {
    "video_name": [],
    "iou": [],
    "conf": [],
    "bird_count": [],
    "drone_count": [],
    "0": [],
    "1": [],
    "2": [],
    "3": [],
    "4": [],
    "5": [],
    "6": [],
    "7": [],
    "8": [],
    "9": [],
    "10": [],
}

for video in MULTISCALE_DATA:
    video_name = video["video_name"]
    print(f"Video: {video_name}")
    class_count = {
        "bird": 0,
        "drone": 0,
    }

    bboxes_count = {}
    
    for idx, frame in enumerate(video["frames"]):
        for label in frame["labels"]:
            if label == 0:
                class_count["bird"] += 1
            else:
                class_count["drone"] += 1
        bboxes_count[len(frame["boxes"])] = bboxes_count.get(len(frame["boxes"]), 0) + 1
    
    print(f"Class counts - Birds: {class_count['bird']}, Drones: {class_count['drone']}")

    
    data["video_name"].append(video_name)
    data["iou"].append(video["iou_threshold"])
    data["conf"].append(video["conf_threshold"])
    data["bird_count"].append(class_count["bird"])
    data["drone_count"].append(class_count["drone"])
    for idx in range(11):
        if idx not in bboxes_count:
            bboxes_count[idx] = 0
        else:
            print(f"Number of frames with {idx} bounding boxes: {bboxes_count[idx]}")
        data[f"{idx}"].append(bboxes_count.get(idx, 0))

    

Video: V_BIRD_043
Class counts - Birds: 307, Drones: 0
Number of frames with 0 bounding boxes: 3
Number of frames with 1 bounding boxes: 307
Video: V_BIRD_045
Class counts - Birds: 303, Drones: 5
Number of frames with 1 bounding boxes: 308
Video: V_BIRD_047
Class counts - Birds: 425, Drones: 3
Number of frames with 0 bounding boxes: 12
Number of frames with 1 bounding boxes: 192
Number of frames with 2 bounding boxes: 118
Video: V_BIRD_050
Class counts - Birds: 301, Drones: 0
Number of frames with 0 bounding boxes: 12
Number of frames with 1 bounding boxes: 283
Number of frames with 2 bounding boxes: 6
Number of frames with 3 bounding boxes: 2
Video: V_BIRD_051
Class counts - Birds: 238, Drones: 50
Number of frames with 0 bounding boxes: 61
Number of frames with 1 bounding boxes: 244
Number of frames with 2 bounding boxes: 22
Video: V_DRONE_099
Class counts - Birds: 4, Drones: 309
Number of frames with 1 bounding boxes: 305
Number of frames with 2 bounding boxes: 4
Video: V_DRONE_101
C

In [25]:
dataframe = pd.DataFrame(data)

In [26]:
dataframe

,video_name,iou,conf,bird_count,drone_count,0,1,2,3,4,5,6,7,8,9,10
0,V_BIRD_043,0.1,0.1,307,0,3,307,0,0,0,0,0,0,0,0,0
1,V_BIRD_045,0.1,0.1,303,5,0,308,0,0,0,0,0,0,0,0,0
2,V_BIRD_047,0.1,0.1,425,3,12,192,118,0,0,0,0,0,0,0,0
3,V_BIRD_050,0.1,0.1,301,0,12,283,6,2,0,0,0,0,0,0,0
4,V_BIRD_051,0.1,0.1,238,50,61,244,22,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,V_DRONE_104,0.8,0.6,22,70,208,92,0,0,0,0,0,0,0,0,0
572,V_DRONE_105,0.8,0.6,3,54,247,57,0,0,0,0,0,0,0,0,0
573,V_DRONE_108,0.8,0.6,0,65,239,65,0,0,0,0,0,0,0,0,0
574,V_DRONE_111,0.8,0.6,0,78,244,78,0,0,0,0,0,0,0,0,0


In [27]:
dataframe.to_csv("outputs/tunning/multiscale_summary.csv", index=False)